# 8.1 계층적 색인

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    1.567350
   2    0.216499
   3    1.268296
b  1   -0.729624
   3    0.433613
c  1    0.571112
   2   -0.466839
d  2   -0.325039
   3    0.327939
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data['b']

1   -0.729624
3    0.433613
dtype: float64

In [5]:
data['b': 'c']

b  1   -0.729624
   3    0.433613
c  1    0.571112
   2   -0.466839
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1   -0.729624
   3    0.433613
d  2   -0.325039
   3    0.327939
dtype: float64

In [8]:
data.loc[:, 2]  # 하위 계층의 객체 선택 가능

a    0.216499
c   -0.466839
d   -0.325039
dtype: float64

In [9]:
data.unstack()  # DataFrame 객체에 unstuck 매서드를 이용하면 데이터를 새롭게 배열 가능

,1,2,3
a,1.567350,0.216499,1.268296
b,-0.729624,NaN,0.433613
c,0.571112,-0.466839,NaN
d,NaN,-0.325039,0.327939


In [10]:
data.unstack().stack()  # unstack의 역수행

a  1    1.567350
   2    0.216499
   3    1.268296
b  1   -0.729624
   3    0.433613
c  1    0.571112
   2   -0.466839
d  2   -0.325039
   3    0.327939
dtype: float64

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index = [['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                            columns=[['Ohio', 'Ohio', 'Colorado'],
                                    ['Green', 'Red', 'Green']])
frame  # DataFrame은 두 축 모두 계층 색인 가능

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [12]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [13]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

- 8.1.1 계층의 순서를 바꾸고 정렬하기

In [14]:
frame.swaplevel('key1', 'key2')  # 계층 순서를 바꿀 수 있음. 하지만 원본 데이터는 바뀌지 않음

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [15]:
frame.sort_index(level = 1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [16]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

- 8.1.2 계층별 요약 통계

In [17]:
frame.sum(level = 'key2')  # key1 의 값으로 나누어진 값 모두 나눈다

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [18]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

- 8.1.3 DataFrame의 컬럼 사용하기

In [20]:
frame1 = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                           'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame1

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [21]:
frame2 = frame1.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [23]:
frame1.set_index(['c', 'd'], drop =False)  # 옆처럼 명시해두어야 column 에서 사라지지 않는다

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [24]:
frame2.reset_index() # 계층 색인이 컬럼으로 이동한다

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
